In [1]:
import pandas as pd

df_points = pd.read_csv('../data/final_df_points_18_21_class.csv')

In [2]:
# add hex enc
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df_points['hex_enc']= label_encoder.fit_transform(df_points['hex_id'])

In [3]:
def on_station(row):
    if row['end_station_number'] != 0:
        return 1
    else: return 0

df_points['on_station'] = df_points.apply(on_station,axis=1)

In [4]:
zone_encoder = preprocessing.LabelEncoder()
df_points['zone_name_enc']= zone_encoder.fit_transform(df_points['zone_name'])

In [5]:
df_points = df_points.reindex(columns=['bike_id', 'lat', 'lng', 'dt_start', 'dt_end', 'hex_id','hex_enc', 'in_zone', 'zone_name','zone_name_enc','end_station_number','on_station','temp', 'rain', 'snow', 'wind_speed', 'humidity', 'year','month', 'day', 'start_min', 'over_night', 'idle_time_next_day', 'idle_time', 'idle_time_class'])

df_points = df_points.sort_values(by=['dt_start'])
df_points

,bike_id,lat,lng,dt_start,dt_end,hex_id,hex_enc,in_zone,zone_name,zone_name_enc,...,wind_speed,humidity,year,month,day,start_min,over_night,idle_time_next_day,idle_time,idle_time_class
0,3694,51.323619,12.335887,1514764029,1514764799,881f1a8d91fffff,324,1,Pinke Zone Leipzig West 1€,4,...,7.20,76.0,2017,12,6,1427,1,821,12,1
1,2508,51.323619,12.335887,1514764035,1514764799,881f1a8d91fffff,324,1,Pinke Zone Leipzig West 1€,4,...,7.20,76.0,2017,12,6,1427,1,1439,12,1
2,41851,51.332415,12.395158,1514764638,1514764799,881f1a1659fffff,137,1,Pinke Zone Leipzig Ost 1€,3,...,7.20,76.0,2017,12,6,1437,1,1439,2,1
3,3694,51.323619,12.335887,1514764800,1514814103,881f1a8d91fffff,324,1,Pinke Zone Leipzig West 1€,4,...,9.30,76.0,2018,1,0,0,0,0,821,4
4,2508,51.323619,12.335887,1514764800,1514851199,881f1a8d91fffff,324,1,Pinke Zone Leipzig West 1€,4,...,9.30,76.0,2018,1,0,0,1,878,1439,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543825,72058,51.317569,12.382730,1640897593,1640904368,881f1a1609fffff,102,1,Blaue Zone Leipzig Ost 0€,1,...,8.94,83.0,2021,12,3,1253,0,0,112,2
2543826,71949,51.334711,12.356763,1640897835,1640899343,881f1a1649fffff,130,1,Pinke Zone Leipzig Ost 1€,3,...,8.94,83.0,2021,12,3,1257,0,0,25,1
2543827,97785,51.367276,12.369792,1640898086,1640900985,881f1a8c8bfffff,265,1,Blaue Zone Leipzig Ost 0€,1,...,8.05,84.0,2021,12,3,1261,0,0,48,1
2543828,41738,51.366942,12.370322,1640900918,1640903516,881f1a8c8bfffff,265,1,Pinke Zone Leipzig Ost 1€,3,...,8.05,84.0,2021,12,3,1309,0,0,43,1


In [6]:
df_points.to_csv('../data/final_df_points_18_21_class_new_features.csv', index=False)

# test

In [7]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import wandb

idle_time_data = pd.read_csv('../data/final_df_points_18_21_class_new_features.csv')

TargetVariable = ['idle_time_class']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'wind_speed', 'humidity', 'dt_start',
              'hex_enc', 'start_min', 'year', 'month', 'day', 'on_station', 'in_zone', 'zone_name_enc']

X = idle_time_data[Predictors].values
y = idle_time_data[TargetVariable].values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=False)


In [13]:
model = RandomForestClassifier()

In [14]:
model.fit(X_train, y_train.ravel())
y_pred = model.predict(X_test)

In [22]:
from sklearn.metrics import f1_score


def eval_classification(y_test,y_pred):
    # Metrics
    # Accuracy, precision, recall
    acc = accuracy_score(y_test, y_pred.ravel())
    macro_precision = precision_score(y_test.ravel(), y_pred.ravel(), average='macro', labels=[1,2,3,4])
    micro_precision = precision_score(y_test.ravel(), y_pred.ravel(), average='micro', labels=[1,2,3,4])
    macro_recall = recall_score(y_test.ravel(), y_pred.ravel(), average='macro', labels=[1,2,3,4])
    micro_recall = recall_score(y_test.ravel(), y_pred.ravel(), average='micro', labels=[1,2,3,4])

    macro_f1 = f1_score(y_test.ravel(), y_pred.ravel(), average='macro', labels=[1,2,3,4])
    micro_f1 = f1_score(y_test.ravel(), y_pred.ravel(), average='micro', labels=[1,2,3,4])

    print(acc)
    print(macro_precision, micro_precision)
    print(macro_recall,micro_recall)
    print(macro_f1,micro_f1)

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred))
accuracy_score(y_test,y_pred)

eval_classification(y_test,y_pred)

print(classification_report(y_test,y_pred))

[[33858 20010  9030  3702]
 [21166 31043 11948  4297]
 [10758 13473 28085 10860]
 [ 3757  7131  9597 35668]]
0.5057492049390093
0.5132548667690033 0.5057492049390093
0.510402563276704 0.5057492049390093
0.5115194559053415 0.5057492049390093
              precision    recall  f1-score   support

           1       0.49      0.51      0.50     66600
           2       0.43      0.45      0.44     68454
           3       0.48      0.44      0.46     63176
           4       0.65      0.64      0.64     56153

    accuracy                           0.51    254383
   macro avg       0.51      0.51      0.51    254383
weighted avg       0.51      0.51      0.51    254383

